In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import sys
import re
import logging
import os
import re
from pathlib import Path
from functools import partial, cache, cached_property
from itertools import chain
from inspect import signature
from uuid import uuid4

import blpapi
import ipywidgets as w
import ipyvuetify as v
import numpy as np
import pandas as pd
import QuantLib as ql
import talib as ta 
from IPython.display import display
from ipydatagrid import DataGrid, TextRenderer, BarRenderer, Expr, VegaExpr
from bqplot import LinearScale, ColorScale, OrdinalColorScale, OrdinalScale
from pandas.tseries.offsets import BDay
from plotly import graph_objects as go
from plotly.subplots import make_subplots
# from bhsysapi import BHSysApi

from api.data.base import data_path, radar_path
from api.gui.params import bbg_params, spot_params, fwd_params
from api.instruments.swap import RealSwap, RollingSwap, Fly, Spread
from api.instruments.utils import parse_dates, parse_number, ez_bump
from api.instruments.strategies import FutureStrategy
from api.instruments.signals import MACDHistCrossover, ZScoreThreshold, EnsambleSignal
from api.instruments.cache import data_cache
from api.instruments.future import FutureChain, FutureSpread, FutureFly
from api.gui.theme import bg_color, grid_style
from api.gui.pandas import DFOutput, negative_red, custom_styler_row_line, DataGridGio
from api.instruments import config as c

from gioutils import ezutils as ez
from gioutils.ezutils import bh, bhs
from gioutils.blpw import BlpQuery
from gioutils.gui.pandas import millify
from gioutils.utils import (
    RepeatedTimer, 
    parse_swap_tenor_expr, 
    parse_offset,
    today, 
    get_next_n, 
    get_n_from_code,
    date_from_offset, 
    get_bday,
    get_bbg_fut_chain_ticker
)

from techi import api as ti

In [3]:
pd.options.plotting.backend = "plotly"
pd.options.display.float_format = millify
c.USE_T_COST = True

In [4]:
bq = BlpQuery(timeout=50000).start()
# ba = BHSysApi(password="A&3jg5Aj")

In [5]:
instrument = FutureChain('rx1', notional='d250', bq=bq)

In [6]:
buy_threshold = -0.7
sell_threshold = 0.7
slope_threshold_buy = 0
slope_threshold_sell = 0

In [9]:
res = ti.MACD(instrument.quote)
res['slope'] = res['macd'].diff()
res['slope_chg'] = (np.sign(res['slope'] * res['slope'].shift()) < 0).astype(int)
buy = ((res['macd'] <= buy_threshold) & (res['slope_chg'] == 1) & (res['slope'] >= slope_threshold_buy)).astype(int)
sell = - ((res['macd'] >= sell_threshold) & (res['slope_chg'] == 1) & (res['slope'] <= slope_threshold_sell)).astype(int)
res['signal'] = buy + sell
leading_nan = res.apply(pd.Series.first_valid_index).max()
res.loc[:res.index[res.index.get_loc(leading_nan) - 1], ['slope_chg', 'signal']] = np.nan

In [11]:
from api.instruments.signals2 import BaseSignal, MACDCustomSignal

In [12]:
signal = MACDCustomSignal(
    instrument=instrument,
    signal_threshold_buy=-0.7,
    signal_threshold_sell=0.7,
    slope_threshold_buy=0,
    slope_threshold_sell=0
)

In [13]:
df = signal.data

In [14]:
index = True

In [17]:
from ipyaggrid import Grid

In [19]:
column_defs = [{'headerName':col,'field':col} for col in df]
if index:
    column_defs = [*[{'headerName':col,'field':col} for col in df.index.names], *column_defs]
grid_options = {
    'defaultColDef': {
        'resizable': True,
    },
    'columnDefs' : column_defs,
    'enableSorting': True,
    'enableFilter': True,
    'enableColResize': True,
    'enableRangeSelection': True,
}

g = Grid(
    grid_data=df,
    grid_options=grid_options,
    quick_filter=True,
    show_toggle_edit=True,
    # export_mode="buttons",
    # export_csv=True,
    # export_excel=True,
    theme='ag-theme-balham',
    # show_toggle_delete=True,
    columns_fit='auto',
    index=index,
    keep_multiindex=True
)
g

Grid(columns_fit='auto', compress_data=True, export_mode='disabled', height='350px', menu={'buttons': [], 'inp…

In [35]:
signals = signal.signals

In [36]:
first_signal_date = (signals.dropna() != 0).idxmin()

In [ ]:
first_signal_index = signals.index

In [33]:
signals

Timestamp('2022-01-18 00:00:00')

In [ ]:
res.to_clipboard()

In [ ]:
pd.DataFrame(np.random.normal(size=(10, 10)))

In [ ]:
res[35:]

In [ ]:
res.to_clipboard()

In [ ]:
bq.bdib(
    'rx1 Comdty', 
    event_type='TRADE',
    start_datetime='2020-01-01',
    end_datetime='2022-12-01',
    interval=5,
)

In [ ]:
df = pd.concat(
    [
        self.quote,
        ti.EMA(self.quote, 12),
        ti.EMA(self.quote, 26),
        ma.ti,
        ma.line.diff(),
    ], 
    axis=1
)

In [ ]:
es = EnsambleSignal(
    instrument=self,
    signals=[
        MACDSignal(),
        ZScoreThreshold()
    ],
    method='sum2'
)


In [ ]:
c = ((self.quote - self.quote.rolling(60).mean()) / self.quote.rolling(60).std()).abs() 

In [ ]:
c.iloc[:c.index.get_loc(c.first_valid_index())]

In [ ]:
self.quote.zscore(window=60, rolling=True)[:60]

In [ ]:
sum

In [ ]:
es.signals.first_valid_index()

In [ ]:
params = {**signature(pd.DataFrame.ewm).parameters, **signature(pd.DataFrame.rolling).parameters}

In [ ]:
kwargs = {}

In [ ]:
fn_kwargs = {}
for p, v in params.items():
    if p != 'self':
        fn_kwargs[p] = kwargs.pop(p, v.default)

In [ ]:
(self.bars.abs() >= 95).astype(int)

In [ ]:
v.default

In [ ]:
@pd.api.extensions.register_series_accessor("zscore")
@pd.api.extensions.register_dataframe_accessor("zscore")
class RollingZScoreZScore:
    def __init__(self, pandas_obj):
        self._obj = pandas_obj

    def __call__(self, *args, **kwargs):
        obj = self._obj
        ewm = kwargs.pop('ewm', False)
        rolling = kwargs.pop('rolling', False)
        accessor = 'rolling' if rolling else 'ewm' if ewm else None
        if accessor:
            accessor = getattr(obj, accessor)
            accessor_ = accessor(*args, **kwargs)
        else:
            accessor_ = obj
        return (obj - accessor_.mean()) / accessor_.std()

In [ ]:
self.bars.zscore(10, rolling=True)

In [ ]:
(self.bars - self. bars.rolling(10).mean()) / self.bars.rolling(10).std()

In [ ]:
hist = '5y'

In [ ]:
self = FutureStrategy(
    signal=MACDSignal(
        instrument=FutureFly.from_legs(
            ticker='ed',
            legs=[2, 6, 10],
            notional=1000,
            start_hist=hist
        ),
    ),

    trade_eop=False,
)

In [ ]:
self = FutureStrategy(
    signal=MACDSignal(
        instrument=FutureFly.from_legs(
            ticker='sfr',
            legs=[2, 6, 10],
            notional=1000,
            start_hist=hist
        ),
    ),
    # sl=0.1,
    # tp=0.1,
    trade_eop=False,
)

In [ ]:
def _any(n=1):
    pass
    


class EnsambleSignal:
        '''
    It has to be the same instrument
    methods:
        all
        any{n}
        mean
        average
    '''
    
    def __init__(self, signals, instrument=None, method=None):
        
        if instrument:
            for signal in signals:
                signal.set_instrument(instrument)
        self.signals = signals
        self.method = method
        
    def signal(self):
        if isinstance(method, str):
            if method[:3] == 'any':
                n = 1 if len(method) == 3 else n[3:]
                method = method[:3]
                
    @property
    def signals(self):
        return pd.concat([signal.signal for signal in self.signals], axis=1)
        

In [ ]:
self.pv.plot()

In [ ]:
check = pd.concat(
    [
        ff.pv.rename('pv'), 
        ff.pv.diff().rename('diff'), 
        ms.line.rename('line'),
        ms.signal.rename('signal'),
        pnl.rename('pnl'),
        t_costs.rename('tcosts'),
        ff.quote,
        ff.quote.diff(),
    ], 
    axis=1
)

In [ ]:
check.to_clipboard()

In [ ]:
ms.signal

In [ ]:
ms.trades

In [ ]:
ms.trades

In [ ]:
ms.line.to_clipboard()
ms.signal.to_clipboard()

In [ ]:
ms.signal.to_clipboard()

In [ ]:
ms.signal.fillna(0).diff().to_clipboard()

In [ ]:
ms.signal.diff().fillna(0).to_clipboard()

In [ ]:
ff.pv.to_clipboard()

In [ ]:
f = FutureChain('ty1')

In [ ]:
f.tick_value * f.notional

In [ ]:
FutureChain('tu1').tick_value

In [ ]:
_.to_clipboard()

In [ ]:
FutureChain('ty1').quote

In [ ]:
FutureChain('tu1').tick_size

In [ ]:
FutureChain('tu1').point_value / 100

In [ ]:
FutureChain('ty1').point_value / 100

In [ ]:
t = ba.getTable(
    namespace="BH_Query", 
    table="query_cont_bars", 
    symbol="EDc1", 
    bar_size="1day", 
    start_date=pd.to_datetime("2022-08-01"),
    roll_adjustment='multiplicative'
)

In [ ]:
catalog = ba.getCatalog()

In [ ]:
catalog

In [ ]:
ba.getTable("BH_SecMaster_Futures", 'roll_offset_mapping').to_clipboard()

In [ ]:
d = _

In [ ]:
d.loc[d['description'].str.contains('SOFR')]

In [ ]:
ba.getTable("BH_SecMaster_Futures", 'main', ticker='SFRU3')

In [ ]:
ba.getTable("BH_SecMaster_Futures", 'main', sector_sub_type='Index', rec_type='FUTURE')

In [ ]:
d = _

In [ ]:
s.trade

In [ ]:
trace1 = s1.pv.plot()

In [ ]:
trace1

In [ ]:
trace2 = swap_rate.plot()

In [ ]:
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    trace1.data[0],
    secondary_y=False,
)

fig.add_trace(
    trace2.data[0],
    secondary_y=True,
)